In [162]:
from labeler import *

data = getLabels("JawClench_labels_Ansh_12-02-21-1918.txt")
startTime = data[0]
labels = data[1]
data = getData("OpenBCI-RAW-2021-12-02_19-19-53.txt",labels,startTime)

data2 = getLabels("JawClench_labels_Ansh_12-02-21-1930.txt")
startTime = data2[0]
labels = data2[1]
data2 = getData("OpenBCI-RAW-2021-12-02_19-31-12.txt", labels, startTime)

data = np.append(data, data2, axis=0).tolist()

2021-12-02 19:19:55.073396
2021-12-02 19:31:14.177762


In [163]:
import numpy as np
from sklearn import svm
for i in range (len(data)):
    for j in range(len(data[i])):
        if j < len(data[i])-1:
            data[i][j] = (float)(data[i][j])
        else:
            data[i][j] = (int)(data[i][j])
data = np.array(data)
print(data[1700])

[1.71241043e+03 6.74333881e+02 9.04597652e+02 1.91071489e+03
 1.00000000e+00]


In [164]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [165]:
def get_batches(data, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(data)//(batch_size_total)
    data = data[:n_batches*batch_size_total, :]
    X = data[:,:-1]
    Y = np.array(data[:, -1], dtype=np.int32)
    X = X.reshape((batch_size, n_batches*seq_length, 4))
    Y = Y.reshape((batch_size, -1))

    for n in range(0, Y.shape[1], seq_length):
        x = X[:, n:n+seq_length]
        y = Y[:, n:n+seq_length]
        yield x, y

In [166]:
batches = get_batches(data, 8, 50)
x, y = next(batches)

In [167]:
print('x\n', x[:1, :10])
print('\ny\n', y[:1, :10])

x
 [[[1615.87426451  475.56941827  807.67066438 1931.42645332]
  [1614.01179433  474.96355447  813.60588561 1941.3895468 ]
  [1649.01351801  490.22608619  826.41878286 1977.03079338]
  [1651.81844298  479.26443942  812.64847122 1960.68743057]
  [1651.89698088  472.18854871  802.00471595 1941.5915014 ]
  [1609.17610368  491.38171528  826.86383095 1921.12302894]
  [1571.44799293  489.05923741  818.68466975 1940.64156681]
  [1518.60694646  474.27915278  805.15371171 1905.17983543]
  [1512.89611923  491.90156137  819.87769784 1912.76809244]
  [1556.76888561  500.28641707  833.50589327 1925.54359069]]]

y
 [[0 0 0 0 0 0 0 0 0 0]]


In [168]:
class LSTMNet(nn.Module):
    
    def __init__(self, num_nodes, num_outputs, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        self.num_nodes = num_nodes
        self.num_outputs=num_outputs
        
        self.lstm = nn.LSTM(num_nodes, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        self.dropout = nn.Dropout(drop_prob)
        
        self.fc = nn.Linear(n_hidden, num_outputs)
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        r_output, hidden = self.lstm(x, hidden)
        
        out = self.dropout(r_output)
        
        out = out.contiguous().view(-1, self.n_hidden)
        
        out = self.fc(out)
        
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

In [169]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = net.num_outputs
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            # x = one_hot_encode(x, n_chars)
            x = np.array(x, dtype=np.float32)
            inputs = torch.from_numpy(x).float()
            targets =  torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # print(targets)
            # print(targets.shape, output.shape)
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    # y = one_hot_encode(y, 2)
                    x, y = torch.from_numpy(x).float(), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [170]:
n_hidden=128
n_layers=3

net = LSTMNet(4, 2, n_hidden, n_layers, drop_prob=0.1)
print(net)

LSTMNet(
  (lstm): LSTM(4, 512, num_layers=2, batch_first=True, dropout=0.1)
  (dropout): Dropout(p=0.1, inplace=False)
  (fc): Linear(in_features=512, out_features=2, bias=True)
)


In [179]:
batch_size = 16
seq_length = 555
n_epochs = 100 # start smaller if you are just testing initial behavior
print(len(data))
# train the model
train(net, data, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10, val_frac=0.0)

27059
Epoch: 4/100... Step: 10... Loss: 0.6709... Val Loss: nan
Epoch: 7/100... Step: 20... Loss: 0.5742... Val Loss: nan
Epoch: 10/100... Step: 30... Loss: 0.5510... Val Loss: nan
Epoch: 14/100... Step: 40... Loss: 0.5516... Val Loss: nan
Epoch: 17/100... Step: 50... Loss: 0.6214... Val Loss: nan
Epoch: 20/100... Step: 60... Loss: 0.5430... Val Loss: nan
Epoch: 24/100... Step: 70... Loss: 0.6051... Val Loss: nan
Epoch: 27/100... Step: 80... Loss: 0.5185... Val Loss: nan
Epoch: 30/100... Step: 90... Loss: 0.4888... Val Loss: nan
Epoch: 34/100... Step: 100... Loss: 0.5148... Val Loss: nan
Epoch: 37/100... Step: 110... Loss: 0.4383... Val Loss: nan
Epoch: 40/100... Step: 120... Loss: 0.4664... Val Loss: nan
Epoch: 44/100... Step: 130... Loss: 0.5022... Val Loss: nan
Epoch: 47/100... Step: 140... Loss: 0.5445... Val Loss: nan
Epoch: 50/100... Step: 150... Loss: 0.4526... Val Loss: nan
Epoch: 54/100... Step: 160... Loss: 0.5763... Val Loss: nan
Epoch: 57/100... Step: 170... Loss: 0.5056...